<table>
    <tr>
        <td class="bTop" colspan="3">
        <img src="logo_UL_FME.png" height="200" width="200">
        </td>
        <td>       
        <img src="LFDT_logo.jpg" height="200" width="200">
        </td>
    </tr>                                           
</table>

# Implementacija algoritma SIMPLE za primer gnane kotanje / <font color='gray'>Implementation of SIMPLE algorithm for lid-driven cavity problem</font>

Avtor: / <font color='gray'> Author: </font> Boštjan Mavrič

## Motivacija / <font color = "gray">Motivation</font>

Rešiti želimo problem gnane kotanje, ki je enostaven primer za primerjavo in verifikacijo metod za reševanje Navier-Stokesovih enačb. Geometrija problema je ilustrirana na spodnji sliki. 

<p>
<font color = "gray">
We want to solve the lid-driven cavity problems, which is a simple case used to compare and verify metods for solution of Navier-Stokes equations. The problem geometry is illustrated in the figure below.
</font>
</p>

<img src="Lid_driven_cavity.png" height="300" width="300">

Problem je definiran z sledečimi brezdimenzijskimi enačbami za komponenti hitrosti $(u, v)$ ter tlak $p$.

<p>
<font color = "gray">
The problem is defined with the following dimensionless equations for velocity components $(u, v)$ and pressure $p$.
</font>
</p>

$$u\frac{\partial u}{ \partial x} + v\frac{\partial u}{\partial y} = - \frac{\partial p}{ \partial x} + \frac{1}{Re}\nabla^2 u$$
$$u\frac{\partial v}{ \partial x} + v\frac{\partial v}{\partial y} = - \frac{\partial p}{ \partial x} + \frac{1}{Re}\nabla^2 v$$

$$\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}=0$$

## Tlačni popravek / <font color = "gray">Pressure correction</font>

Ker problem gnane kotanje ni sklopljen z okolico prek tlaka, je potrebno sistem tlačnih enačb regularizirati. V algoritmu SIMPLE dobimo enačbo za tlak prek pogoja, da ima popravljeno hitrostno polje divergenco enako nič. V zveznih spremenljivkah je to enačba, ki povezuje Laplaceov operator na tlačnem polju $p$ z divergenco poskusne hitrosti $(u^*, v^*)$.

<p>
<font color = "gray">
Because the lid-driven cavity problem is not coupled to the environment through pressure, the system of equations for $p$ needs to be regularized. In the SIMPLE algorithm, the pressure equation is obtained through the condition that the crrected velocity field is divergence-free. In continuous variables this gives an equation that connects Laplace operator on pressure with divergence of guess velocity $(u^*, v^*)$.
</font>
</p>

$$ \nabla^2 p = \frac{\partial u^*}{\partial x} + \frac{\partial v^*}{\partial y}$$.

Na robu imamo Neumannove robne pogoje za tlak, ki preprečujejo tok tekočine v steno.

<p>
<font color = "gray">
At the boundary we have Neumann pressure boundary conditions, which prevent the fluid to penetrate the walls.
</font>
</p>


$$\frac{\partial p}{\partial n}=g$$

Ta sistem enačb je degeneriran, saj tlačno polje $p$ ni enolično določeno. Če imamo polje $p$, ki reši parcialno diferencialno enačbo in robne pogoje, potem polje $p'=p+C$, kjer je $C$ poljubna konstanta, tudi rešitev, saj v enačbi nastopajo prvi in drugi odvodi $p$, ki jih dodana konstanta $C$ ne spremeni. Ob numeričnem reševanju sistema enačb lahko tako dobimo rešitev, kjer je vrednost konstante $C$ tako visoka, da zaradi končne natančnosti računalnika ni mogoče dovolj točno izračunati odvoda $p$, ki je potreben za izračun popravljenega hitrostnega polja. 

<p>
<font color = "gray">
This system of equations is non-regular, since the pressure field $p$ is not uniquely defined. If we have filed $p$, which solves the PDE and the boundary conditions, them the field $p=p'+C$, where $C$ is an arbitrary constant, is also a solution since the equations are given in terms of first and second derivatives, which are not affected by additional constant $C$. When numerically solving the system of discretized pressure equations, we can obtain solution with such a high value of the constant $C$, that we cannot accurately calculate the corrected velocity field because of limited accuracy of computer architecture.
</font>
</p>


Da to težavo odpravimo, zahtevamo še, da je povprečna vrednost tlačnega polja v kotaniji enaka 0, da pa ohranimo sistem rešljiv je potrebno uvesti Lagrangeov multiplikator $\alpha$. S tem je sistem regularen saj s tem pogojem enolično določimo vrednost konstante $C$, vrednost $\alpha$ pa določimo prek diskretiziranega sistema enačb in ima vrednost velikosti diskretizacijske napake. Nov, regulariziran sistem enačb je tako:

<p>
<font color = "gray">
To remedy this problem, we additionaly require that the average value of pressure in the cavity equals 0. To keep the system solvable, we interoduce lagrange multiplicator $\alpha$. This regularizes the system, since it uniquely determines the value of constant $C$. The value of $\alpha$ is determined when solving the system of equations and should be of the same magnitude as the discretization error. New, regularized system of equations is:
</font>
</p>

$$\nabla^2 p = \frac{\partial u^*}{\partial x} + \frac{\partial v^*}{\partial y} - \alpha$$

$$\int_V p dV=0$$

V diksretiziranih enačbah se uvedena regularizacija odraža v dodatni vrstici in stolpcu napolnjenima z vrednostjo $1$, ki sta dodana sistemu enačb dobljenih iz SIMPLE algoritma $P$. Nov sistem linearnih enačb je v bločni obliki:

<p>
<font color = "gray">
In discretized equations the introduced regularization is reflected in an additional row and column filled with values 1, that are added to the matrix system $P$ obtained through SIMPLE algorithm. In block form the system of equations is given in block form as:
</font>
</p>

$$ \begin{bmatrix} & & &1\\ \quad & P & \quad & 1 \\ & & &1\\1& 1& 1 & 0\end{bmatrix} \begin{bmatrix} \\ p \\ \\ \alpha\end{bmatrix} = \begin{bmatrix} \\ \frac{\partial u^*}{\partial x} + \frac{\partial v^*}{\partial y} \\ \\ 0\end{bmatrix} $$.


<p>
<font color = "gray">

</font>
</p>

In [ ]:
#SIMPLE_demo, a demonstration of fluid flow solver using SIMPLE algorithm 
#Copyright (C) 2020  Boštjan Mavrič
#
#This program is free software: you can redistribute it and/or modify
#it under the terms of the GNU General Public License as published by
#the Free Software Foundation, either version 3 of the License, or
#(at your option) any later version.
#
#This program is distributed in the hope that it will be useful,
#but WITHOUT ANY WARRANTY; without even the implied warranty of
#MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#GNU General Public License for more details.
#
#You should have received a copy of the GNU General Public License
#along with this program.  If not, see <http://www.gnu.org/licenses/>.

!export MKL_DEBUG_CPU_TYPE=5 # Remove Intel cheats on AMD hardware

from __future__ import division
import matplotlib.pyplot as plt
import math as mt
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sp_linalg

from IPython.display import display, clear_output

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

## Sistem shranjevanja vrednosti polj  / <font color = "gray">System of field value storage</font>

Problem se rešuje na zamaknjeni mreži končnih volumnov predstavljenih na spodnji sliki. Vrednosti tlaka so podane v zelenih točkah, vrednosti $u$ v točkah z rdečo puščico ter vrednosti $v$ v točkah z modro puščico. Z istimi barvami so označeni tudi ustrezni kontrolni volumni, ki jih uporabimo za generiranje diskretiziranih vodilnih enačb.

<p>
<font color = "gray">
The problem is being solved on staggered finite volume grid shown on the figure below. The pressure values are given in green points, the values of $u$ in points with blue arrow and the values of $u$ in the points with blue arrow. The same colors are used also to denote corresponding control volumes that are used to generate discretized governing equations.
</font>
</p>

<img src="FVM_stagg_grid.png" height="400" width="400">

Vrednosti polj za takšno mrežo lahko neposredno shranimo v matriko. Če je vzdolž ene osi $N$ volumnov, potem za shranjevanja tlačnih vrednosti potrebujemo matriko velikosti $N\times N$. Iz skice je razvidno da za vrednosti $u$ potrebujemo dodatni stolpec, za vrednosti $v$ pa dodatno vrstico. Vrednosti $u$ tako lahko shranimo v matriko velikosti $(N+1) \times N$, vrednosti $v$ pa v $N\times (N+1)$. Tako shranjene vrednosti so primerno urejene za izračun desnih strani enačb. 

<p>
<font color = "gray">
Field values can be stored directly in a matrix for such a grid. If we position $N$ control volumes along one axis, then we need matrix of size $N \times N$ to store pressure values. From the illustration we can see that we nneds an extra column to store values of $u$ and an extra row to store values $v$. This means that the values of $u$ can be stored in a matrix of size $(N+1) \times N$, and values of $v$ in a matrix of size $N\times (N+1)$. Such values are ordered in a convenient manner to calculate elements of the right-hand-side vectors.
</font>
</p>

Za reševanje sistemov enačb, ki so podane z matrikami, pa moramo spremenljivke $p$, $u$ in $v$ urediti v dolge vektorje. To naredimo tako da vrstice iz matrik z vrednostmi polj v vektor postavimo  eno za drugo v dolge vektorje. Te preslikave podatkov in indeksov med obe načinoma shranjevanja so implementirane v spodnjih funkcijah.

<p>
<font color = "gray">
To solve systems of equations, which are given in matrix form, we need to store values of $p$, $u$ and $v$ in long vectors. We do this by placing the rows of the matrices with field values sequentially one by one in the long vectors. The mappings between data and indices of the two storage schemes are implemented in the functions below.
</font>
</p>

In [ ]:
# Preslikava med indeksi premaknjene mreže kontrolnih volumnov na dolge vektorje za reševanje sistema linearnih enačb.
# / Mapping of the indices from staggered grid to the "long" vectors used to solve systems of linear equations.
def glob_ind_u(i, j, N):
    return (i) + j*(N+1)

def glob_ind_v(i, j, N):
    return (i) + j*N

def glob_ind_p(i, j, N):
    return (i) + j*N

# Interpolacija hitrostnih vrednosti na položaje tlačnih točk.
# / Interpolate velocity values to the positions of pressure variables.
def interpolate_to_pressure_pos(us, vs, vec, N):
    for i in range(0, N):
        for j in range(0, N):
            vec[0, i, j]=(us[i, j]+ us[i+1, j])/2
            vec[1, i, j]=(vs[i, j]+ vs[i, j+1])/2

# Preslikaj  vrednosti iz dolgih vektorjev na matrike, ki ustrezajo zamaknjeni mreži montrolnih volumov. /
# / Map values from "long" vectors to the arrays that correspond to staggered grid.
def reshuffle_us(u_sol, us, N):
    for i in range(0, N+1):
        for j in range(0, N):
            us[i, j]=u_sol[glob_ind_u(i, j, N)]

def reshuffle_vs(v_sol, vs, N):
    for i in range(0, N):
        for j in range(0, N+1):
            vs[i, j]=v_sol[glob_ind_v(i, j, N)]

def reshuffle_p(p_sol, ps, N):
    for i in range(0, N):
        for j in range(0, N):
            ps[i, j]=p_sol[glob_ind_p(i, j, N)]


## Implementacija algoritma simple  / <font color = "gray">System of field value storage</font>

In [ ]:
# Funkcija, ki reši sistem enačb.
# / Main function used to solve the system of equations.
def SIMPLE_demo(N, FD_kind, Re, alpha_p, alpha_vel):
    Nmax_iters=200
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,6))
    
    p_corr_norm=np.zeros(Nmax_iters)
    v_corr_norm=np.zeros(Nmax_iters)
    plt.ion()
    plt.show()
    
    ###
    # Shranjevanje spremenljivk.
    # / Storage for unknowns.

    # Shramba za u. 
    # / Storage of u.
    # u ima eno vrstico spremenljivk več vzdolž x kot vzdolž y.
    # / For u, there is one more row of variables along x axis than along y.
    Nall_u=(N)*(N+1)
    us=np.zeros([N+1, N])
    us_old=np.zeros([N+1, N])
    ds_u=np.zeros([N+1, N])
    rhs_u=np.zeros(Nall_u)
    u_sol=np.zeros(Nall_u)

    # Shramaba za v
    # / Storage of v.
    # v ima eno eno vrstico spremenljivk vzdolž y osi kot vzdolž x.
    # / There is one more row of v variables along y axis than along x.
    Nall_v=(N+1)*(N)
    vs=np.zeros([N, N+1])
    vs_old=np.zeros([N, N+1])
    ds_v=np.zeros([N, N+1])
    rhs_v=np.zeros(Nall_v)
    v_sol=np.zeros(Nall_v)

    # Shramba za p.
    # / Storage of p.
    Nall_p=(N)*(N)
    ps=np.zeros([N, N])
    vel_at_p=np.zeros([2, N, N])
    vel_norm_at_p=np.zeros([N, N])
    ps_prime=np.zeros([N, N])

    # Za stabilizacijo tlačnega popravka potrebujemo eno enačbo več kot je tlačnih vrednosti.
    # / To stabilize pressure correction equation we need on more unknown for regularization of the system.
    rhs_p=np.zeros(Nall_p+1)       
    p_sol=np.zeros(Nall_p+1)
    p_old=np.zeros(Nall_p+1)

    # Shranjevanje matrik.
    # / Storage for matrices.
    # Redke matrike sestavimo v obliki "seznama seznamov". Preden jih uporabimo v reševalniku jih spremenimo v format CSR.
    # / We use sparse matrices in "List-of-lists" for construction. Before using them in the solver, they are converted to CSR format.
    sparse_u=sp.lil_matrix((Nall_u, Nall_u))
    sparse_v=sp.lil_matrix((Nall_v, Nall_v))
    sparse_p=sp.lil_matrix((Nall_p+1, Nall_p+1))

    # Generirajmo zamaknjeno mrežo.
    # / Generation of staggered grid.
    h=1./(N)

    # Točke z vrednostjo u.
    # / Points with known value of u.
    points_u=np.zeros([2, Nall_u])
    for i in range(0, N+1):
        for j in range(0, N):
            Ip=glob_ind_u(i, j, N)
            points_u[0, Ip]=h*i
            points_u[1, Ip]=h/2 + h*j

    # Točke z vrednostjo v.
    # / Points with known value of v.
    points_v=np.zeros([2, Nall_v])
    for i in range(0, N):
        for j in range(0, N+1):
            Ip=glob_ind_v(i, j, N)
            points_v[0, Ip]=h/2 + h*i
            points_v[1, Ip]=h*j

    # Točke z vrednostjo tlaka.        
    # / Points with known value of p.        
    points_p=np.zeros([2, Nall_p])
    points_p_contour=np.zeros(N)
    for i in range(0, N):
        points_p_contour[i]=h/2 + h*i
        for j in range(0, N):
            Ip=glob_ind_p(i, j, N)
            points_p[0, Ip]=h/2 + h*i
            points_p[1, Ip]=h/2 + h*j
    
    
    rho=1e0
    eta=1e0/Re
    v_lid=1e0

    # Izraze poenostavimo za konstantne vrednosti gostote in transportnih koeficientov.
    # / The expressions are simplified by assuming constant density and constant transport coefficients.
    Gamma=eta
    
    # Difuzijski koeficienti iz V&M učbenika.
    # / Diffusion coefficents from V&M book.
    Dw=Gamma/h
    De=Gamma/h
    Ds=Gamma/h
    Dn=Gamma/h

    A=h

    # Začetek iteracijske zanke.
    # / Start of the the iteration loop.
    for itr in range(0, Nmax_iters):
        Pe_u=0.
        
        ###################################
        # Pripravi sistem enačb za poskusno hitrost (u, v).
        # / Set up the system of equations for guess velocity (u, v).
        ###################################
        
        ###################################
        # u-komponenta hitrosti, notranjosti.
        # / u-component of velocity, interior.
        for i in range(1, N):
            for j in range(1, N-1):
                # Določi potrebne indekse točk.
                # / Determine indices of points under consideration.
                Ip=glob_ind_u(i, j, N)
                Ie=glob_ind_u(i+1, j, N)
                Iw=glob_ind_u(i-1, j, N)
                In=glob_ind_u(i, j+1, N)
                Is=glob_ind_u(i, j-1, N)

                # Določi advektivne tokove.
                # / Determine advective fluxes.
                Fw=rho*(us[i, j] + us[i-1, j])/2
                Fe=rho*(us[i+1, j] + us[i, j])/2
            
                Fs=rho*(vs[i, j] + vs[i-1, j])/2
                Fn=rho*(vs[i, j+1] + vs[i-1, j+1])/2

                # Določi koeficiente v linearnem sistemu enačb.
                # / Determine the coefficents used in linear systems of equations.
                if(FD_kind=="central"):
                    a_w=Dw+Fw/2
                    a_e=De-Fe/2
                    a_s=Ds+Fs/2
                    a_n=Dn-Fn/2
                elif(FD_kind=="upwind"):
                    a_w=Dw+max([Fw, 0.])
                    a_e=De+max([-Fe, 0.])
                    a_s=Ds+max([Fs, 0.])
                    a_n=Dn+max([-Fn, 0.])
                elif(FD_kind=="hybrid"):
                    a_w=max([Fw, (Dw+Fw/2), 0])
                    a_e=max([-Fe, (De-Fe/2), 0])
                    a_s=max([Fs, (Ds+Fs/2), 0])
                    a_n=max([-Fn, (Dn-Fn/2), 0])

                a_p=a_w+a_e+a_s+a_n +(Fe-Fw)+(Fn-Fs)

                # Nadzorujmo Pecletovo število.
                # / Monitor the Peclet number.
                Pe_u=max([Pe_u, max([Fw, Fe, Fs, Fn])/Dw])
                
                # Nastavimo matrične elemente.
                # / Populate the matrix elements.
                sparse_u[Ip, In]=-a_n
                sparse_u[Ip, Iw]=-a_w
                sparse_u[Ip, Ip]=a_p
                sparse_u[Ip, Ie]=-a_e
                sparse_u[Ip, Is]=-a_s

                # Shranimo vrednosti za tlani popravek.
                # / Store values for pressure correction.
                ds_u[i, j]=h/a_p
            
                # Nastavimo vrednost desne strani za u komponento.
                # / Set up RHS for u-component.
                rhs_u[Ip]=A*(ps[i-1, j] - ps[i, j])

        # u-kompnenta hitrosti v točkah v stiku s severno/južno steno.
        # / u-component of velocity in cells in contact with north/south walls.
        for i in range(1, N):
            for j in [0, N-1]:
                
                Ip=glob_ind_u(i, j, N)
                Ie=glob_ind_u(i+1, j, N)
                Iw=glob_ind_u(i-1, j, N)
                In=glob_ind_u(i, j+1, N)
                Is=glob_ind_u(i, j-1, N)
                
                Fw=rho*(us[i, j] + us[i-1, j])/2
                Fe=rho*(us[i+1, j] + us[i, j])/2
            
                Fs=rho*(vs[i, j] + vs[i-1, j])/2
                Fn=rho*(vs[i, j+1] + vs[i-1, j+1])/2

                if(FD_kind=="central"):
                    a_w=Dw+Fw/2
                    a_e=De-Fe/2
                    a_s=Ds+Fs/2
                    a_n=Dn-Fn/2
                elif(FD_kind=="upwind"):
                    a_w=Dw+max([Fw, 0.])
                    a_e=De+max([-Fe, 0.])
                    a_s=Ds+max([Fs, 0.])
                    a_n=Dn+max([-Fn, 0.])
                elif(FD_kind=="hybrid"):
                    a_w=max([Fw, (Dw+Fw/2), 0])
                    a_e=max([-Fe, (De-Fe/2), 0])
                    a_s=max([Fs, (Ds+Fs/2), 0])
                    a_n=max([-Fn, (Dn-Fn/2), 0])

                sparse_u[Ip, Iw]=-a_w
                sparse_u[Ip, Ie]=-a_e

                # Stena pri y=0.
                # / Wall at y=0.
                if(j==0):
                    a_s=a_s+Ds
                    a_p=a_w+a_e+a_s+a_n +(Fe-Fw)+(Fn-Fs)
                    a_p=a_p
                    sparse_u[Ip, Ip]=a_p
                    sparse_u[Ip, In]=-a_n

                    # Nastavimo vrednost v vektorju desnih strani.
                    # / Set up RHS for u-component.
                    rhs_u[Ip]=A*(ps[i-1, j] - ps[i, j])
                    
                # Stena pri y=1, kjer je gibajoč pokrov.
                # / Wall at y=1, the position of moving lid.
                elif(j==N-1):
                    a_n=a_n+Dn
                    a_p=a_w+a_e+a_s+a_n + (Fe-Fw)+(Fn-Fs)
                    a_p=a_p
                    sparse_u[Ip, Ip]=a_p
                    sparse_u[Ip, Is]=-a_s

                    # Nastavimo desne strani za u-komponento. Dodatni člen povzroči gibanje tekočine.
                    # / Set up RHS for u-component, the last term provides the driving.
                    rhs_u[Ip]=A*(ps[i-1, j] - ps[i, j]) + a_n*v_lid 
                
                ds_u[i, j]=h/a_p               
            
        # u kompnenta hitrosti v celicah na stiku z vzhodno/zahodno steno.
        # / u-component of velocity in cells in contact with east/west walls.
        for i in [0, N]:
            for j in range(0, N):

                Ip=glob_ind_u(i, j, N)

                # Dirichletovi robni pogoji.
                # / Dirichlet boundary conditions.
                sparse_u[Ip, Ip]=1
            
                ds_u[i, j]=0
            
                # Nastavimo vrednosti v vektorju desnih strani za u-komponento.
                # / Set up RHS for u-component.
                rhs_u[Ip]=0

        # Ponovimo postopek še za v-komponento.
        # / The same procedure for v-component.
        ###################################
        # v-komponenta hitrosti , notranjost.
        # v-component of velocity, interior.
        for i in range(1, N-1):
            for j in range(1, N):
                
                Ip=glob_ind_v(i, j, N)
                Ie=glob_ind_v(i+1, j, N)
                Iw=glob_ind_v(i-1, j, N)
                In=glob_ind_v(i, j+1, N)
                Is=glob_ind_v(i, j-1, N)
            
                Fw=rho*(us[i, j] + us[i, j-1])/2
                Fe=rho*(us[i+1, j] + us[i+1, j-1])/2
            
                Fs=rho*(vs[i, j-1] + vs[i, j])/2
                Fn=rho*(vs[i, j] + vs[i, j+1])/2

                if(FD_kind=="central"):
                    a_w=Dw+Fw/2
                    a_e=De-Fe/2
                    a_s=Ds+Fs/2
                    a_n=Dn-Fn/2
                elif(FD_kind=="upwind"):
                    a_w=Dw+max([Fw, 0.])
                    a_e=De+max([-Fe, 0.])
                    a_s=Ds+max([Fs, 0.])
                    a_n=Dn+max([-Fn, 0.])
                elif(FD_kind=="hybrid"):
                    a_w=max([Fw, (Dw+Fw/2), 0.])
                    a_e=max([-Fe, (De-Fe/2), 0.])
                    a_s=max([Fs, (Ds+Fs/2), 0.])
                    a_n=max([-Fn, (Dn-Fn/2), 0.])
                
                a_p=a_w+a_e+a_s+a_n +(Fe-Fw) +(Fn-Fs)

                sparse_v[Ip, In]=-a_n
                sparse_v[Ip, Iw]=-a_w
                sparse_v[Ip, Ip]=a_p
                sparse_v[Ip, Ie]=-a_e
                sparse_v[Ip, Is]=-a_s

                ds_v[i, j]=h/a_p

                # Nastavimo vrednost v vektorju desnih strani za v-komponento.
                # / Set up RHS for v-component.
                rhs_v[Ip]=A*(ps[i, j-1] - ps[i, j])

        # v-komponenta hitrosti v celicah v stiku z vzhodno/zahodno steno.
        # / v-component of velocity in cells in contact with east/west walls.
        for i in [0, N-1]:
            for j in range(1, N):
            
                Ip=glob_ind_v(i, j, N)
                Ie=glob_ind_v(i+1, j, N)
                Iw=glob_ind_v(i-1, j, N)
                In=glob_ind_v(i, j+1, N)
                Is=glob_ind_v(i, j-1, N)
            
                Fw=rho*(us[i, j] + us[i-1, j])/2
                Fe=rho*(us[i+1, j] + us[i, j])/2
            
                Fs=rho*(vs[i, j] + vs[i-1, j])/2
                Fn=rho*(vs[i, j+1] + vs[i-1, j+1])/2

                if(FD_kind=="central"):
                    a_w=Dw+Fw/2
                    a_e=De-Fe/2
                    a_s=Ds+Fs/2
                    a_n=Dn-Fn/2
                elif(FD_kind=="upwind"):
                    a_w=Dw+max([Fw, 0.])
                    a_e=De+max([-Fe, 0.])
                    a_s=Ds+max([Fs, 0.])
                    a_n=Dn+max([-Fn, 0.])
                elif(FD_kind=="hybrid"):
                    a_w=max([Fw, (Dw+Fw/2), 0])
                    a_e=max([-Fe, (De-Fe/2), 0])
                    a_s=max([Fs, (Ds+Fs/2), 0])
                    a_n=max([-Fn, (Dn-Fn/2), 0])

                sparse_v[Ip, In]=-a_n
                sparse_v[Ip, Is]=-a_s
            
                if(i==0):
                    a_w=a_w + Dw
                    a_p=a_w+a_e+a_s+a_n +(Fe-Fw)+(Fn-Fs)
                    a_p=a_p 
                    sparse_v[Ip, Ip]=a_p
                    sparse_v[Ip, Ie]=-a_e
                elif(i==N-1):
                    a_e=a_e + De
                    a_p=a_w+a_e+a_s+a_n +(Fe-Fw)+(Fn-Fs)
                    a_p=a_p
                    sparse_v[Ip, Ip]=a_p
                    sparse_v[Ip, Iw]=-a_w
                
                ds_v[i, j]=h/a_p
            
                # Nastavi vrednosti v vektorju desnih strani za v-komponento.
                # / Set up RHS for v-component.
                rhs_v[Ip]=A*(ps[i, j-1] - ps[i, j])

        # v-komponenta hitrosti v celicah v stiku s severno/južno steno.
        # / v-component of velocity in cells in contact with north/south walls.
        for i in range(0, N):
            for j in [0, N]:

                Ip=glob_ind_v(i, j, N)
            
                sparse_v[Ip, Ip]=1
            
                ds_v[i, j]=0

                # Nastavi vrednosti v vektorju desnih strani za u-komponento.
                # / Set up RHS for u-component.
                rhs_v[Ip]=0

        # Rešimo sistem enačb za u.
        # / Solve the system of equations for u.
        u_sol=sp_linalg.spsolve(sparse_u.tocsr(), rhs_u)
        reshuffle_us(u_sol, us, N)

        # Rešimo sistem enačb za v.
        # / Solve the system of equations for v.
        v_sol=sp_linalg.spsolve(sparse_v.tocsr(), rhs_v)
        reshuffle_vs(v_sol, vs, N)
        
        # Sestavimo tlačno matriko.
        # / Assemble the pressure matrix.
        for i in range(0, N):
            for j in range(0, N):
                Ip=glob_ind_p(i, j, N)
                Ie=glob_ind_p(i+1, j, N)
                Iw=glob_ind_p(i-1, j, N)
                In=glob_ind_p(i, j+1, N)
                Is=glob_ind_p(i, j-1, N)

                a_e=ds_u[i+1, j]/h
                a_w=ds_u[i, j]/h
                a_n=ds_v[i, j+1]/h
                a_s=ds_v[i, j]/h

                # Obravnavamo robne pogoje.
                # / Treat the boundary conditions.
                if(i==0) :
                    a_w=2*a_w
                else:
                    sparse_p[Ip, Iw]=-a_w

                if(i==N-1):    
                    a_e=2*a_e
                else:
                    sparse_p[Ip, Ie]=-a_e
                
                if(j==0):
                    a_s=2*a_s
                else:
                    sparse_p[Ip, Is]=-a_s
                
                if(j==N-1):
                    a_n=2*a_n
                else:
                    sparse_p[Ip, In]=-a_n
                
                a_p=a_e+a_w+a_n+a_s
                sparse_p[Ip, Ip]=a_p
            
                # Izračunajmo vrednosti v vektorju desnih strani.
                # / Determine values in RHS vector for pressure.
                rhs_p[Ip]=h*(us[i,j] - us[i+1,j] + vs[i,j] - vs[i,j+1])   

        # Dodamo regularizacijsko enačbo.
        # / Add the regularization equation sum(p)=0.
        rhs_p[Nall_p]=0
        sparse_p[0:Nall_p, Nall_p]=1e0
        sparse_p[Nall_p, 0:Nall_p]=1e0

        # Rešimo sistem enačb.
        # / Solve the system of equations.
        p_old=p_sol.copy()
        p_sol=sp_linalg.spsolve(sparse_p.tocsr(), rhs_p)
        p_vals=p_sol[0:Nall_p]

        reshuffle_p(p_sol, ps_prime, N)
        p_corr_norm[itr]=np.linalg.norm(ps_prime)/(np.linalg.norm(ps))

        # Posodobimo vrednosti tlaka.
        # / Update pressure.
        ps=ps+alpha_p*ps_prime
        
        # Popravimo vrednosti u.
        # / Correct u values.
        for i in range(1, N):
            for j in range(0, N):
                us[i, j]=us[i, j] + ds_u[i, j]*(ps_prime[i-1, j]-ps_prime[i, j])

        v_corr_norm[itr]=np.linalg.norm(us-us_old)/(np.linalg.norm(us_old))
        
        # Popravimo vrednosti v.
        # / Correct v values.
        for i in range(0, N):
            for j in range(1, N):
                vs[i, j]=vs[i, j] + ds_v[i, j]*(ps_prime[i, j-1]-ps_prime[i, j])

        # Podrelaksacija popravkov hitrosti.
        # / Under-relax velocity corrections.
        us=alpha_vel*us + (1.-alpha_vel)*us_old
        us_old=us.copy()
        
        vs=alpha_vel*vs + (1.-alpha_vel)*vs_old
        vs_old=vs.copy()

        # Priprava podatkov za izris.
        # / Extract data for plotting.
        interpolate_to_pressure_pos(us, vs, vel_at_p, N)
        for i in range(0, N):
            for j in range(0, N):
                vel_norm_at_p[i, j]=np.linalg.norm(vel_at_p[:, i, j])

        pmin=np.amin(ps)
        pmax=np.amax(ps)

        ax1.cla()
        ax2.cla()
        ax3.cla()

        ax1.set_title("Streamlines and pressure field")
        
        ax1.contourf(points_p_contour, points_p_contour, ps.transpose(), levels=np.linspace(pmin, pmax, 30), cmap='YlGnBu_r')
        ax1.streamplot(points_p[0, 0:N], points_p[1, 0:Nall_p:N], vel_at_p[0, :, :].transpose(), vel_at_p[1, :, :].transpose(), color=vel_norm_at_p.transpose(), density=1, linewidth=3., cmap='hot')

        ax1.set_aspect('equal')
        ax1.set_xlim(0, 1)
        ax1.set_ylim(0, 1)
        
        ax2.set_title("Relative magnitude of corrections")
        ax2.set_yscale('log')
        press_line=ax2.plot(p_corr_norm[1:itr], label="Pressure")
        vel_line=ax2.plot(v_corr_norm[1:itr], label="Velocity")
        ax2.legend()

        ax3.plot(points_p_contour, vs[:, int(N/2)+1], label="v @ y=0.5")
        ax3.plot(points_p_contour, us[int(N/2)+1, :], label="u @ x=0.5")
        ax3.set_title("Velocity profiles")
        ax3.legend()
        #plt.draw()
        clear_output(wait=True)
        display(fig)
        #fig.show()
        plt.pause(.0001)
        

    # Izriši končno rešitev.
    # / Plot final solution.    
    plt.ioff()    
    plt.clf()
    plt.title("Final velocity")
    plt.axes().set_aspect('equal')
    plt.streamplot(points_p[0, 0:N], points_p[1, 0:Nall_p:N], vel_at_p[0, :, :].transpose(), vel_at_p[1, :, :].transpose(), color=vel_norm_at_p.transpose(), density=3)
    plt.show()
    
    pmin=np.amin(ps)
    pmax=np.amax(ps)
    
    plt.clf()
    plt.title("Final pressure")
    plt.axes().set_aspect('equal')
    plt.contourf(points_p_contour, points_p_contour, ps.transpose(), levels=np.linspace(pmin, pmax, 20))
    plt.show()

In [ ]:
def wrapper_for_visualization(N, alpha_p, alpha_v, Re, uw_type):
    SIMPLE_demo(N, uw_type, Re, alpha_p, alpha_v)

N_slider = widgets.IntSlider(min=10, max=100, step=10, value=20)
Re_slider = widgets.FloatSlider(min=1, max=1000, step=50, value=100)
alpha_p_slider = widgets.FloatSlider(min=0.01, max=1., step=0.1, value=0.9)
alpha_v_slider = widgets.FloatSlider(min=0.01, max=1., step=0.1, value=0.9)

uw_inter = widgets.Dropdown(options=['central','upwind', 'hybrid'], 
                           value='central', 
                           description='Upwinding scheme') 

w=widgets.interactive(wrapper_for_visualization,
                      N=N_slider,
                      alpha_p=alpha_p_slider,
                      alpha_v=alpha_v_slider,
                      Re=Re_slider,
                      uw_type=uw_inter)


display(w)